In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms 
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:


print(torch.__version__)
print(torchvision.__version__)


1.13.0+cu117
0.14.0+cu117


In [3]:
import os
folder_path = 'C:/Users/siddu/SU22_project/cursive_prediction/datafileszipped/datafiles'
os.chdir(folder_path)
files_in_folder = os.listdir()

In [4]:
file_dic = {}
for file in files_in_folder:
    letter = file[-5]
    file_dic[file] = letter

In [5]:
df = pd.DataFrame(list(file_dic.items()), columns=['file_name', 'letter'])

In [6]:
df

,file_name,letter
0,0000_a.png,a
1,0001_b.png,b
2,0002_n.png,n
3,0003_o.png,o
4,0004_e.png,e
...,...,...
1935,1935_i.png,i
1936,1936_x.png,x
1937,1937_o.png,o
1938,1938_l.png,l


In [7]:

IMG_SIZE = 64
BATCH_SIZE = 32

In [8]:
from PIL import Image

def png_to_tensor(path):
    # Replace 'path_to_image.png' with the actual path to your PNG file
    image_path = folder_path + '/' + path

    # Open the image
    image = Image.open(image_path)
    bw_image = image.convert('L')
    data_transforms = [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(), # Scales data into [0,1] 
        transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1] 
    ]
    data_transform = transforms.Compose(data_transforms)
    img_tensor = data_transform(bw_image)


    return img_tensor

In [9]:
# FUNCTION TO ADD NOISE 
def add_gaussian_noise(tensor, mean=0.0, std=0.1):
    noise = torch.randn(tensor.size()) * std + mean
    noisy_tensor = tensor + noise
    return torch.clamp(noisy_tensor, min=-1.0, max=1.0)

In [10]:
# IMAGE TO NOISY TENSOR
def png_to_noisy_tensor(path, noise_mean=0.0, noise_std=0.1):
    # Replace 'path_to_image.png' with the actual path to your PNG file
    image_path = folder_path + '/' + path

    # Open the image
    image = Image.open(image_path)
    bw_image = image.convert('L')
    data_transforms = [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(), # Scales data into [0,1] 
        transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1] 
    ]
    data_transform = transforms.Compose(data_transforms)
    img_tensor = data_transform(bw_image)

    # Add Gaussian noise to the tensor
    noisy_img_tensor = add_gaussian_noise(img_tensor, mean=noise_mean, std=noise_std)

    return noisy_img_tensor


In [11]:
def show_tensor_image(tensor):
    reverse_transforms = transforms.Compose([
        transforms.Lambda(lambda t: (t + 1) / 2),
        transforms.Lambda(lambda t: t.permute(1, 2, 0)), # CHW to HWC
        transforms.Lambda(lambda t: t * 255.),
        transforms.Lambda(lambda t: t.numpy().astype(np.uint8)),
        transforms.ToPILImage(),
    ])
    return reverse_transforms(tensor)

In [12]:
df2 = df.copy()

In [13]:
df2

,file_name,letter
0,0000_a.png,a
1,0001_b.png,b
2,0002_n.png,n
3,0003_o.png,o
4,0004_e.png,e
...,...,...
1935,1935_i.png,i
1936,1936_x.png,x
1937,1937_o.png,o
1938,1938_l.png,l


In [14]:
df['image'] = df['file_name'].apply(png_to_tensor)
df.drop(columns = ['file_name'], inplace = True)
df = df[['image', 'letter']]

In [15]:
df['image'][0]

tensor([[[0.5608, 0.5843, 0.6627,  ..., 0.6784, 0.6392, 0.5922],
         [0.5373, 0.5843, 0.6549,  ..., 0.6706, 0.6392, 0.6000],
         [0.4980, 0.5529, 0.6078,  ..., 0.6471, 0.6235, 0.6000],
         ...,
         [0.7333, 0.7255, 0.7020,  ..., 0.7333, 0.7647, 0.7725],
         [0.7176, 0.7255, 0.7176,  ..., 0.7412, 0.7647, 0.7725],
         [0.7176, 0.7255, 0.7255,  ..., 0.7412, 0.7647, 0.7725]]])

In [16]:
df['image'][0]

tensor([[[0.5608, 0.5843, 0.6627,  ..., 0.6784, 0.6392, 0.5922],
         [0.5373, 0.5843, 0.6549,  ..., 0.6706, 0.6392, 0.6000],
         [0.4980, 0.5529, 0.6078,  ..., 0.6471, 0.6235, 0.6000],
         ...,
         [0.7333, 0.7255, 0.7020,  ..., 0.7333, 0.7647, 0.7725],
         [0.7176, 0.7255, 0.7176,  ..., 0.7412, 0.7647, 0.7725],
         [0.7176, 0.7255, 0.7255,  ..., 0.7412, 0.7647, 0.7725]]])

In [17]:
for i in range(10):
    noise_df = df2.copy()
    noise_df['image'] = noise_df['file_name'].apply(png_to_noisy_tensor)
    noise_df.drop(columns = ['file_name'], inplace = True)
    noise_df = noise_df[['image', 'letter']]    
    df = df.append(noise_df, ignore_index= True)


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader


# df to tuples
data_tuples = list(zip(df['image'].tolist(), df['letter'].tolist()))

# Split indices
dataset_size = len(data_tuples)
indices = list(range(dataset_size))
split = int(0.2 * dataset_size)

torch.manual_seed(42)

# Shuffle indices
indices_shuffled = torch.randperm(len(indices))

# Split 
train_indices, test_indices = indices_shuffled[split:], indices_shuffled[:split]

# custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        image_tensor, letter = self.data[index]
        return image_tensor, ord(letter) - ord('a')

    def __len__(self):
        return len(self.data)

# custom Datasets
train_dataset = CustomDataset(data=[data_tuples[i] for i in train_indices])
test_dataset = CustomDataset(data=[data_tuples[i] for i in test_indices])

#  DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [19]:
for X_batch, y_batch in test_loader:
    print("X batch:", X_batch)
    print("y batch:", y_batch)
    break  

X batch: tensor([[[[1.0000, 0.8657, 0.9670,  ..., 0.9781, 0.9129, 1.0000],
          [1.0000, 0.8868, 0.9270,  ..., 0.9154, 1.0000, 0.9166],
          [0.8755, 1.0000, 1.0000,  ..., 0.8742, 1.0000, 1.0000],
          ...,
          [1.0000, 0.7935, 1.0000,  ..., 0.9009, 1.0000, 0.9045],
          [0.8716, 1.0000, 0.7441,  ..., 0.9688, 0.9644, 1.0000],
          [0.9646, 0.8707, 0.6269,  ..., 1.0000, 0.9671, 0.9071]]],


        [[[0.8766, 1.0000, 0.7122,  ..., 1.0000, 1.0000, 1.0000],
          [0.9496, 0.9385, 1.0000,  ..., 0.9002, 0.9505, 1.0000],
          [0.9890, 0.9430, 0.9324,  ..., 1.0000, 0.9005, 0.8240],
          ...,
          [0.7166, 0.8107, 0.6624,  ..., 0.8010, 0.8060, 0.9374],
          [0.8903, 0.9505, 0.9509,  ..., 1.0000, 0.8886, 0.9253],
          [0.9944, 1.0000, 1.0000,  ..., 0.8692, 0.9616, 1.0000]]],


        [[[0.6321, 0.5455, 0.5228,  ..., 0.8175, 0.7731, 0.6978],
          [0.5864, 0.4454, 0.5503,  ..., 0.7412, 0.6202, 0.6066],
          [0.6491, 0.7377, 0.

In [20]:
len(train_loader)

534

In [21]:
# AlexNet architecture
class AlexNet(nn.Module):
    def __init__(self, num_classes=26):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [22]:
#model
alexnet = AlexNet().to(device)

#loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.01, momentum=0.9)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Training Finished")


C:\Users\siddu\AppData\Local\Temp\ipykernel_9504\1123731909.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1, Loss: 2.7722884606332814
Epoch 2, Loss: 1.5896464276514697
Epoch 3, Loss: 0.5629587309428034
Epoch 4, Loss: 0.2928874114674799
Epoch 5, Loss: 0.17587311396615976
Epoch 6, Loss: 0.12047956879983206
Epoch 7, Loss: 0.11709189361332147
Epoch 8, Loss: 0.029267835329748217
Epoch 9, Loss: 0.02597711741858707
Epoch 10, Loss: 0.03790932806733314
Training Finished


In [23]:

test_loss = 0.0

alexnet.eval()

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        inputs = inputs.to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

    print(f"Test Loss: {test_loss/len(test_loader)}")

print("Testing Finished")


C:\Users\siddu\AppData\Local\Temp\ipykernel_9504\3622266950.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Test Loss: 0.2833216420458213
Testing Finished


In [56]:
def pred_img(img):
    processed_img = png_to_tensor(img).to(device)
    processed_img = processed_img.unsqueeze(0)

    with torch.no_grad():
        alexnet.eval()
        output = alexnet(processed_img)

    return chr(torch.argmax(output) + ord('a'))


In [66]:
test = '1939_r.png'	

In [67]:
out = pred_img(test)

In [68]:
out


'r'